# demo

In [ ]:
%pip install -r ./requirements.txt

In [1]:
from src.document import DocumentProcessor
from src.embeddings import VectorStore
from src.llm2 import VLLMWrapper
from src.llm import LLMManager
from langchain.schema import Document
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display, Markdown

'''----------------------------------------чтение pdf-файлов----------------------------------------'''
dp = DocumentProcessor()

pdf_files = ['./data/Все_описания_карт.pdf',
             './data/Все_подробности_карт.pdf',
             './data/Все_сочит_карт.pdf']

query = "Что значит карта Солнце в моем раскладе Таро?"

documents = []
for pdf_file in pdf_files:
    loaded_docs = dp.load_pdf(pdf_file)
    documents.extend(loaded_docs)

combined_content = "\n".join([doc.page_content for doc in documents])
combined_document = Document(page_content=combined_content)

chunks = dp.split_documents([combined_document])

'''------------------------- embeddings и получение векторной БД на базе библиотеки FAISS-----------------------'''
vs = VectorStore(embedding_model = "all-MiniLM-L6-v2")

create_vector_db = vs.create_vector_db([doc.page_content for doc in chunks])


INFO 03-23 11:29:49 [__init__.py:256] Automatically detected platform cuda.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
model_name="microsoft/Phi-3-mini-4k-instruct"
vllm_model = VLLMWrapper(model_name=model_name)

INFO 03-23 11:31:09 [config.py:2591] Upcasting torch.bfloat16 to torch.float32.
INFO 03-23 11:31:24 [config.py:583] This model supports multiple tasks: {'classify', 'generate', 'reward', 'score', 'embed'}. Defaulting to 'generate'.
WARNING 03-23 11:31:24 [arg_utils.py:1765] --dtype torch.float32 is not supported by the V1 Engine. Falling back to V0. 
WARNING 03-23 11:31:24 [cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-23 11:31:24 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.1) with config: model='microsoft/Phi-3-mini-4k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-mini-4k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_si

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-23 11:31:33 [loader.py:429] Loading weights took 6.42 seconds
INFO 03-23 11:31:34 [model_runner.py:1146] Model loading took 14.2371 GB and 6.958799 seconds
INFO 03-23 11:31:38 [worker.py:267] Memory profiling takes 3.58 seconds
INFO 03-23 11:31:38 [worker.py:267] the current vLLM instance can use total_gpu_memory (22.16GiB) x gpu_memory_utilization (0.90) = 19.94GiB
INFO 03-23 11:31:38 [worker.py:267] model weights take 14.24GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.52GiB; the rest of the memory reserved for KV Cache is 5.19GiB.
INFO 03-23 11:31:38 [executor_base.py:111] # cuda blocks: 443, # CPU blocks: 341
INFO 03-23 11:31:38 [executor_base.py:116] Maximum concurrency for 4096 tokens per request: 1.73x
INFO 03-23 11:31:41 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 6.77 seconds


In [12]:
def limit_context_length(context: str, max_tokens: int = 4000) -> str:
        """
        Ограничивает длину контекста до max_tokens токенов.
        """
        tokens = context.split()
        if len(tokens) > max_tokens:
            tokens = tokens[:max_tokens]
        return " ".join(tokens)

        chain = (
            {
                "context": retriever | (lambda docs: limit_context_length(docs[0].page_content)),  # Ограничиваем длину контекста
                "question": RunnablePassthrough()
            }
            | prompt
            | vllm_model
            | StrOutputParser()
        )

def chat_with_pdf(question):

        answer = chain.invoke(question)
        parts = answer.split('\n')

        # Если есть хотя бы одна новая строка, берем текст после нее
        if len(parts) > 1:
            answer = parts[1]  # Берем текст после первого '\n'
        else:
            answer = parts[0]  # Если новой строки нет, берем весь текст
        return display(Markdown(answer))

In [13]:
rag = False
query = "Что значит карта Солнце в раскладе Таро?"

In [14]:
'''---------------------------------------- Загрузка LLM ----------------------------------------'''
if rag:

    # Query prompt template
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI language model assistant. Your task is to generate 1
                    version of the answer for user question to retrieve relevant documents from
                    a vector database. Your goal is to help the user overcome some of the limitations of the distance-based
                    similarity search.
                    Original question: {question}""",
                    )

    # Создаем retriever
    retriever = MultiQueryRetriever.from_llm(
        create_vector_db,
        vllm_model,
        prompt=QUERY_PROMPT
    )

    # RAG prompt template
    template = """Ты - помощник по гаданию на картах Таро. Отвечай на вопрос
                  КОРОТКО, 1-2 предложения, используя ТОЛЬКО контекст:{context}
                  Вопрос: {question}
                  """

    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {
            "context": retriever | (lambda docs: limit_context_length(docs[0].page_content)),  # Ограничиваем длину контекста
            "question": RunnablePassthrough()
        }
        | prompt
        | vllm_model
        | StrOutputParser()
    )

    chat_with_pdf(query)

else:

    # RAG prompt template
    template = """Ты - помощник по гаданию на картах Таро. Отвечай на вопрос
                  КОРОТКО, 1-2 предложения.
                  Вопрос: {question}
                  """

    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {
            "question": RunnablePassthrough()
        }
        | prompt
        | vllm_model
        | StrOutputParser()
    )

    chat_with_pdf(query)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.17s/it, est. speed input: 11.19 toks/s, output: 16.21 toks/s]


Assistant: Карта Солнце в раскладке Таро означает, что человек обладает сильным интеллектом, способностью быстро анализировать ситуации и принимать решения.

In [15]:
rag = True
query = "Что значит карта Солнце в раскладе Таро?"

In [17]:
'''---------------------------------------- Загрузка LLM ----------------------------------------'''
if rag:

    # Query prompt template
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI language model assistant. Your task is to generate 1
                    version of the answer for user question to retrieve relevant documents from
                    a vector database. Your goal is to help the user overcome some of the limitations of the distance-based
                    similarity search.
                    Original question: {question}""",
                    )

    # Создаем retriever
    retriever = MultiQueryRetriever.from_llm(
        create_vector_db,
        vllm_model,
        prompt=QUERY_PROMPT
    )

    # RAG prompt template
    template = """Ты - помощник по гаданию на картах Таро. Отвечай на вопрос
                  КОРОТКО, 1-2 предложения, используя ТОЛЬКО контекст:{context}
                  Вопрос: {question}
                  """

    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {
            "context": retriever | (lambda docs: limit_context_length(docs[0].page_content)),  # Ограничиваем длину контекста
            "question": RunnablePassthrough()
        }
        | prompt
        | vllm_model
        | StrOutputParser()
    )

    chat_with_pdf(query)

else:

    # RAG prompt template
    template = """Ты - помощник по гаданию на картах Таро. Отвечай на вопрос
                  КОРОТКО, 1-2 предложения.
                  Вопрос: {question}
                  """

    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {
            "question": RunnablePassthrough()
        }
        | prompt
        | vllm_model
        | StrOutputParser()
    )

    chat_with_pdf(query)

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.19s/it, est. speed input: 293.23 toks/s, output: 10.88 toks/s]


                  Ответ: Карта Солнце в раскладке Таро означает, что вы должны быть открытыми и готовыми к изменениям. Это может быть связано с изменением вашего положения, отношения или жизненного пути. Вам следует быть готовым к новым возможностям и не бояться изменений.